In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist
import scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import average,complete,single,weighted,centroid
import plotly.figure_factory as ff
import scipy.spatial as scs
from scipy.spatial.distance import pdist,squareform
from numpy import savetxt
from matplotlib.ticker import AutoMinorLocator
from matplotlib import gridspec
import plotly.figure_factory as ff
import scipy.cluster.hierarchy as sch
import matplotlib
import seaborn as sns; sns.set()
from scipy.cluster.hierarchy import dendrogram, fcluster, leaves_list,linkage
from scipy.spatial import distance
from scipy.cluster import hierarchy# You can use SciPy one too
import os
import sys
import warnings
import glob
from Bio import SeqIO
import itertools
import numpy as np
from tqdm import tqdm
import time

In [2]:
# Collect the path for .rgi files from user and load
path = '/home/amjad/NICHEnbhd/allrgisrequired/'
readfiles=glob.glob(os.path.join(path,"*.txt"))

In [3]:
# Collect the path for .gbk files from user and load
path = '/home/amjad/NICHEnbhd/allgbksrequired/'
gbkfiles=glob.glob(os.path.join(path,"*.gbk"))

In [4]:
# Function to extract the required data from gbk files(entire genomes:) using biopython library #  
def extract(infile):
  gene_start=[]
  gene_end=[]
  gene_strand=[]
  gene_name=[]
  loc_tag=[]
  function=[]
  protein_seq=[]
  contig_name=[]
  unique=[]

  for index, record in enumerate(SeqIO.parse(infile, "genbank")):
    #print("index %i, ID = %s, length %i, with %i features"% (index, record.id, len(record.seq), len(record.features)))
    for i in record.features:
        if i.type == "CDS" and "gene" in i.qualifiers:
          locations=i.location
          gene_start.append(locations.start)
          gene_end.append(locations.end)
          gene_strand.append(locations.strand)
          loc_tag.append(i.qualifiers['locus_tag'])
          function.append(i.qualifiers['product'])
          protein_seq.append(str(i.qualifiers['translation']))
          gene_name.append(i.qualifiers['gene'])
          contig_name.append(record.id)
        elif i.type =="CDS":
          locations=i.location
          gene_start.append(locations.start)
          gene_end.append(locations.end)
          gene_strand.append(locations.strand)
          loc_tag.append(i.qualifiers['locus_tag'])
          function.append(i.qualifiers['product'])
          protein_seq.append(str(i.qualifiers['translation']))
          gene_name.append("UID")
          contig_name.append(record.id)
          
  salmonella_gene_frame=pd.DataFrame()
  salmonella_gene_frame['GeneStart']=gene_start
  salmonella_gene_frame['GeneEnd']=gene_end
  salmonella_gene_frame['GeneStrand']=gene_strand
  salmonella_gene_frame['Locus_Tag']=loc_tag
  salmonella_gene_frame['GeneName']=gene_name
  salmonella_gene_frame['Product']=function
  salmonella_gene_frame['ProteinSequence']=protein_seq
  salmonella_gene_frame['contig_name']=contig_name
  
  #print(contig_name)
  for i in contig_name:
    if i not in unique:
      unique.append(i)


  return salmonella_gene_frame,unique


In [5]:
# Function to create groups based on contigs #
def make_groups(frame,column):
  group=frame.groupby(frame[column])
  datasets = {}  
  for groups, data in group:
    datasets[groups] = data
  return datasets


In [6]:
#Creating separate dictionary for all the unique AMR genes of the genome as "key" using the field "Best_Hit_ARO" #
def createeachdict_drug(drugindex):
	temp_dict={}
	for j,k in datadict.items():       
	    temp=k[k['Best_Hit_ARO']==drugindex]
	    if len(temp)>0:
	        temp_dict[j]=temp
	return temp_dict


In [7]:
# A function to assign color to all and any AMR genes in the neighborhood based on CARD color codes for strict, loose and perfect hits #
color_Dict={"Loose":"#DC7633 ","Perfect":"#28B463","Strict":"#F4D03F"}
def checkforRGIinneighborhood(e,genome,rgi_gene):
    for q in range(len(e["GeneEnd"])):
        for p in range(len(datadict[genome]["Stop"])):
            datadict[genome].reset_index(drop=True, inplace=True)
            #print(e["GeneEnd"][q],p)
            if e["GeneEnd"][q] == datadict[genome]["Stop"][p] and e["GeneName"][q]==rgi_gene:
                e["Genecolor"][q]=color_Dict[datadict[genome]["Cut_Off"][p]]
                                    
    return e

In [8]:
# A function to delete keys of those AMR genes who are not present in atleast 25% of the total genomes ##
def delete_keys_lessthan_25percent_instances(dict_element):
    total_genomes= len(gbkfiles)
    minimum_genomes = int((total_genomes /25)*100)
    emptykeyslist=[]
    for i,j in dict_element.items():
        if len(j)<=minimum_genomes:
            emptykeyslist.append(i)
    for i in emptykeyslist:
        del dict_element[i]
    return dict_element

In [9]:
# A function to compute neighbors of each AMR gene based on contig info ##
def find_neighbor(rec,uname,data,number_of_genes,drug,genome,key,instancetype):
      
      neighbor_genes=[]
      rec.reset_index(drop=True, inplace=True)
      contig_flag=0

      for j in range(len(rec['Start'])):
            m=[]
            n=[]
            upwardgenes=[]
            downwardgenes=[]
            recarray=[]
            i=rec.loc[j].Start
            w=rec.loc[j].Stop
            k=rec.loc[j].req_cont
            g=number_of_genes


            if k in uname:
                newlist=data[k]
                newlist.reset_index(drop=True, inplace=True)
                for l in range(len(newlist)):
                    if newlist['GeneStart'][l] > i and newlist['GeneEnd'][l] > w:
                        downwardgenes.append((newlist['GeneStart'][l],newlist['GeneEnd'][l],newlist['GeneStrand'][l],newlist['Locus_Tag'][l],newlist['ProteinSequence'][l],newlist['GeneName'][l],"#A9F1EE","Not_Applicable"))
                    else:
                        upwardgenes.append((newlist['GeneStart'][l],newlist['GeneEnd'][l],newlist['GeneStrand'][l],newlist['Locus_Tag'][l],newlist['ProteinSequence'][l],newlist['GeneName'][l],"#A9F1EE","Not_Applicable"))
                    #print(upwardgenes) 

                newu=pd.DataFrame(upwardgenes,columns=["GeneStart","GeneEnd","Strand","Locus_Tag","ProteinSequence","GeneName","Genecolor","Gene_Cut_Off"])
                m=(newu.iloc[(newu['GeneStart']-i).abs().argsort()[:g+1]]).sort_values(by="GeneStart")
                #print(m)
                newd=pd.DataFrame(downwardgenes,columns=["GeneStart","GeneEnd","Strand","Locus_Tag","ProteinSequence","GeneName","Genecolor","Gene_Cut_Off"])
                n=(newd.iloc[(newd['GeneStart']-i).abs().argsort()[:g]]).sort_values(by="GeneStart")
                #print(n)


                recarray.append((rec['Start'][j],rec['Stop'][j],rec['Orientation'][j],rec['Locus_Tag'][j],rec['Predicted_Protein'][j],rec['Best_Hit_ARO'][j],"#ccccff",rec["Cut_Off"][j]))
                o=pd.DataFrame(recarray,columns=["GeneStart","GeneEnd","Strand","Locus_Tag","ProteinSequence","GeneName","Genecolor","Gene_Cut_Off"])


            else:
                print(k,genome)
                print("contig does not exist----"+"drugclass:"+drug)
                print(rec['Best_Hit_ARO'][j])
      
      
      if len(m)!=0 and len(n)==0:
          m.reset_index(drop=True, inplace=True)
          m=m.drop([len(m)-1])
          e = pd.concat([m,o,n], ignore_index=True)
          e.reset_index(drop=True, inplace=True)
          neighbor_genes.append(e)
              
      if len(m)==0 and len(n)!=0:

          m.reset_index(drop=True, inplace=True)
          n=n.drop([len(n)-1])
          e = pd.concat([m,o,n], ignore_index=True)
          e.reset_index(drop=True, inplace=True)
          neighbor_genes.append(e)
         

      if(len(m)!=0 and len(n)!=0):
        m.reset_index(drop=True, inplace=True)
        m=m.drop([len(m)-1])
        e = pd.concat([m,o,n], ignore_index=True)
        e.reset_index(drop=True, inplace=True)
        neighbor_genes.append(e)
            
      return neighbor_genes,contig_flag

In [10]:
def locus_generator(frame,genome_name):
    tag=[]
    RGI_name_array=[]
        
    frame.reset_index(drop=True, inplace=True)
    for index in range(len(frame)):
        temp_name=frame["Best_Hit_ARO"][index].split(" ")
        if len(temp_name)>1:
            name=(temp_name[0][0]+temp_name[1][0]+"_"+temp_name[2])
            RGI_name_array.append(name)
        else:
            name=(temp_name[0])
            RGI_name_array.append(name)
            
        tag.append(genome_name+"("+name+")"+frame["Cut_Off"][index][0]+"_"+str(frame["Best_Identities"][index]))
        
    return tag,RGI_name_array

In [11]:
# A function to get separate dicts of locus tags,protein sequences and gene names inorder to compare and write to a fasta file####
def getrequiredgenes(frame,number_of_genes,drug):
    temp_locus_array=[]
    temp_protein_array=[]
    temp_genename=[]
    #print(drug)
    for i in frame:
        temp_locus_array.append(list(i['Locus_Tag']))
        temp_protein_array.append(i['ProteinSequence'])
        temp_genename.append((i['GeneName']))
    
    
    locus_to_protein_dict={}
    for i in frame:
        for j in range(len(i)):
            
            locus_to_protein_dict[i['Locus_Tag'][j].strip()]=i['ProteinSequence'][j]
  
    a=temp_locus_array
    a=list(itertools.chain.from_iterable(a))
   
    b=temp_protein_array
    b=list(itertools.chain.from_iterable(b))
   
    c=temp_genename
    c=list(itertools.chain.from_iterable(c))
    
 
    if number_of_genes==10:
        if len(frame)>1:
            return(a[:10]+a[-10:], b[:10]+b[-10:],c[:10]+c[-10:])## for more than one card genes in a genomes
        else:
            return a,b,c
        #return(a[:5]+a[-5:], b[:5]+b[-5:],c[:5]+c[-5:])## for more than one card genes in a genomes
    if number_of_genes==14:
        
        if len(frame)>1:
            print(drug)
            return(a[:14]+a[-14:], b[:14]+b[-14:],c[:14]+c[-14:])## for more than one card genes in a genomes
        else:
            return a,b,c

In [12]:
# Read each rgi file(.txt) using pandas and store them in a dataframe#
# Read RGI files but focus only on vanA gene
dataframelist=[]
filenames=[]
datadict={}
for i in sorted(readfiles): 
    dfak=pd.read_csv(i,sep="\t") # added by ak to replace '/' by '-'
    dfak["Best_Hit_ARO"] = dfak["Best_Hit_ARO"].apply(lambda x: x.replace("/", "-"))
    #filenames.append(os.path.basename(i).split(".")[0])
    dfAK=pd.DataFrame(dfak)
    required_gene = dfAK[dfAK["Best_Hit_ARO"] == 'tet(45)']
    if required_gene.empty:
        print('DataFRame is empty')
    else:
        dataframelist.append(pd.DataFrame(required_gene))
        filenames.append(os.path.basename(i).split(".")[0])
    #break

DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is

DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is empty
DataFRame is

In [13]:
# Dictionary of dataframes with keys as filenames ###
for i in tqdm(range(len(filenames))):
    datadict[filenames[i]]=pd.DataFrame(dataframelist[i])
    time.sleep(0.1)

100%|█████████████████████████████████████████| 455/455 [00:46<00:00,  9.73it/s]


In [14]:
len(datadict)

455

In [15]:
# Replacing the orientation of rgi dataframe to match the data in .gbk format to help with comparison #
for i,j in tqdm(datadict.items()):
    j["Orientation"]=j["Orientation"].replace("-","-1")
    j["Orientation"]=j["Orientation"].replace("+","+1")
    time.sleep(0.1)

100%|█████████████████████████████████████████| 455/455 [00:48<00:00,  9.44it/s]


In [16]:
# Adding two new columns 1> Modified Locus tag and 2> contig to extract the right neighbors #
for k,v in datadict.items():
    newcon=[]
    for i in v["Contig"]:
        head,sep,tail=i.partition("_")
        newcon.append(head)
    v['req_cont']=newcon    #adding a new column "req_cont" into dataframe
    v['Locus_Tag'],v["Best_Hit_ARO"]=locus_generator(v,k)

In [17]:
# Read the .gnk file data, store the info in the sorted order in dictionaries #
gbk_names=[]
uniquenames=[]
datasetslist=[]
gbkdict={}
uniquedict={}

In [18]:
#for i in sorted(gbkfiles):    
    #gbk_names.append(os.path.basename(i).split(".")[0])
#for i in gbk_names:
    #uniquenames.append(str(i))
    #datasetslist.append(str(i))
gbk_names=filenames
uniquenames=gbk_names
datasetslist=gbk_names

In [19]:
#gbkfiles=sorted(gbkfiles) #read the files in the sorted order
path = '/home/amjad/NICHEnbhd/allgbksrequired/'
ext ='.gbk'
gbkfiles = (path + pd.Series(filenames) + ext).tolist()

In [20]:
len(datadict)

455

In [21]:
for i in range(len(gbk_names)):
    a=gbk_names[i]
    b=uniquenames[i]
    gbkdict[a],uniquedict[b]=extract(gbkfiles[i])

In [22]:
len(datasetslist)

455

In [23]:
# Convert the Locus tag column from  ['SA200015_0001'] --> SA200015_001 to help while writing to fasta file required to BLAST #
for j,i in gbkdict.items():
    i['Locus_Tag']=i['Locus_Tag'].apply(lambda i:str(i).replace("[","").replace("]","").replace("'",""))
    i['ProteinSequence']=i['ProteinSequence'].str.strip('[]')

In [24]:
# To supress the warning that arise when a single value of dataframe column is manipulated #
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()
pd.set_option('mode.chained_assignment', None)

In [25]:
# The block that causes the chained assignment error #

for i,j in tqdm(gbkdict.items()):
    for value in range(len(j["GeneName"])):
        if isinstance(j["GeneName"].iloc[value], list):
            j["GeneName"].iloc[value] = j["GeneName"].iloc[value][0]
    time.sleep(0.1)

100%|█████████████████████████████████████████| 455/455 [03:32<00:00,  2.14it/s]


In [26]:
# For each .gbk file, divides the data based on available contigs using groupby and dictionary, used to find the neighbors of same contig# 
datasetdict={}
for i in tqdm(range(len(gbk_names))):
    for j,k in gbkdict.items():
        datasetdict[j]=make_groups(k,"contig_name")
    time.sleep(0.1)

100%|█████████████████████████████████████████| 455/455 [17:54<00:00,  2.36s/it]


In [27]:
# Finds the union of all the AMR genes present in more than 25 percent of the total genomes #
uniquedrugdict={}
unionofdrugclasess=[]

for j,k in tqdm(datadict.items()):
    uniquedrugclasses=[]
    for l in range(len(k)):
        if k['Best_Hit_ARO'][l]  not in uniquedrugclasses:
                uniquedrugclasses.append(k["Best_Hit_ARO"][l])
    
    uniquedrugdict[j]=uniquedrugclasses
    time.sleep(0.1)
        
for i,j in tqdm(uniquedrugdict.items()):
    for item in j:
        if item not in unionofdrugclasess:
            unionofdrugclasess.append(item)
    time.sleep(0.1)

100%|█████████████████████████████████████████| 455/455 [00:45<00:00,  9.93it/s]


In [28]:
# Converts the longer AMR gene names into shorter version to make them as dictionary keys and to make it easier to visualize in gene order image#
# Haemophilis influenzea PBP3 --> Hi_PBP3#

listofdrugnames_modified=[]
for k in unionofdrugclasess:
    if len(k.split(" "))>1:
        temp=k.split(" ")
        listofdrugnames_modified.append(temp[0][0]+temp[1][0]+"_"+temp[2])
    else:

        listofdrugnames_modified.append(k.split("; ")[0].split(" ")[0])

In [29]:
# A main dictionary of all the AMR models with all the genomes 
main_dictionary={}
for i in tqdm(range(len(listofdrugnames_modified))):
    main_dictionary[listofdrugnames_modified[i]]=createeachdict_drug(unionofdrugclasess[i])
    time.sleep(0.1)

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.59it/s]


In [30]:
Dict_multigene_instances={}
Dict_singlegene_instances={}
for i,j in tqdm(main_dictionary.items()):
    temp={}
    flag=0
    for a,b in j.items():
        
        if len(b)>1:
            flag=1
    if flag==1:
        Dict_multigene_instances[i]=j
    else:
        Dict_singlegene_instances[i]=j
    time.sleep(0.1)

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  9.85it/s]


In [31]:
dict_rgi_single_occurance={}
g={}
for i,j in Dict_singlegene_instances.items():
    temp={}
    l=[]
    deletekeylist=[]
    for a,b in j.items():
        if len(b)==1:
            l.append(a)
            b.reset_index(drop=True, inplace=True)            
            x=make_groups(b,"Locus_Tag")
            for e,f in x.items():
                temp[e]=f

    dict_rgi_single_occurance[i]=temp
    g[i]=l
 
for i,j in Dict_singlegene_instances.items():
    for t in g[i]:
        del j[t]
        
for i,j in dict_rgi_single_occurance.items():    
     j.update(Dict_singlegene_instances[i])

In [32]:
singleinstance_neighboringdict_combined_range10={}
Single_contig_end_flag_dict={}
for i,j in dict_rgi_single_occurance.items():
    Dict_neighboring_genes_range10={}
    temp_contig_flag={}
    for k,l in j.items():
        key=k.split("(")[0]
        temp,contigflag=find_neighbor(j[k],uniquedict[key],datasetdict[key],10,i,k,key,"Single_Instance_Neighborhood")
        if len(temp)>0:
            Dict_neighboring_genes_range10[k]=temp
            temp_contig_flag[k]=contigflag
    singleinstance_neighboringdict_combined_range10[i]=Dict_neighboring_genes_range10
    Single_contig_end_flag_dict[i]=temp_contig_flag

In [33]:
# Code to extract the locus tags, protein sequences and gene names to use to later for comparison #
Drug_singleinstance_range_10_locus_dict10={}
Drug_singleinstance_range_10_protein_dict10={}
Drug_singleinstance_range_10_genename_dict10={}

for i,j in singleinstance_neighboringdict_combined_range10.items():
    locustags_dict_10={}
    protein_dict_10={}
    genename_dict_10={}
    for k,l in j.items():
        locustags_dict_10[k],protein_dict_10[k],genename_dict_10[k]=getrequiredgenes(j[k],10,i)  
    Drug_singleinstance_range_10_locus_dict10[i]=locustags_dict_10
    Drug_singleinstance_range_10_protein_dict10[i]=protein_dict_10
    Drug_singleinstance_range_10_genename_dict10[i]=genename_dict_10

In [34]:
# Read the .txt files of BLAST results and store in a dictionary ######
filepath_single='/home/amjad/NICHEamrnbhd/Outputs/Single_instance/output_blast_single_instance'
Single_instance_blastdataframelist=[]
Single_instance_blastfilename=[]
readfiles=glob.glob(os.path.join(filepath_single,"*.txt"))

In [35]:
for i in readfiles:    
    Single_instance_blastfilename.append(os.path.basename(i).split(".")[0])
    i=pd.read_csv(i,index_col = False,sep="\t", names=['query_id','sub_id','PI','len','Evalue','bitscore','qseq','sseq'])
    Single_instance_blastdataframelist.append(i)
temp_Single_instance_blastdatadict={}### dictionary of dataframes with keys as filenames ###
for i in range(len(Single_instance_blastfilename)):
    a=Single_instance_blastdataframelist[i]
    b=Single_instance_blastfilename[i]
    temp_Single_instance_blastdatadict[b]=pd.DataFrame(a)

In [36]:
# To sort and filter only those BLAST results with more than 70 percent identity 

Single_instance_blastdatadict={}
for i,j in temp_Single_instance_blastdatadict.items():
    j=j[j["PI"]>70]
    Single_instance_blastdatadict[i]=j

In [37]:
# A function to normalize the bitscore values 
def normalized_bitscore(z): 
  manval=[]
  a=z[z['query_id']==z['sub_id']]
  manval=[]
  a=z[z['query_id']==z['sub_id']]
  a.reset_index(drop=True, inplace=True)
  q={}
  for i in range(len(a)):
    q[a['query_id'][i]]=a['bitscore'][i]

  for i in range(len(z)):
      manval.append(round(float(z['bitscore'][i])/q[z['query_id'][i]],3))

  z['normalized_bitscore']=manval
  return z

In [38]:
Single_instance_drugclass_genomedict={}
for k,l in tqdm(Drug_singleinstance_range_10_locus_dict10.items()):
    u=Single_instance_blastdatadict[k]
    temp_dict={}
    for a,b in l.items():        
        temp=[]
        u.reset_index(drop=True, inplace=True)
        for index in range (len(u)):
            if  u["query_id"][index] in b:
                temp.append((u["query_id"][index],u["sub_id"][index],u["PI"][index],u["bitscore"][index]))
                    
        
        temp_dict[a]=pd.DataFrame(temp,columns=["query_id","sub_id","PI","bitscore"])
    time.sleep(0.1)  
    Single_instance_drugclass_genomedict[k]=temp_dict
    

for i,j in tqdm(Single_instance_drugclass_genomedict.items()):
    for a,b in j.items():
        b=normalized_bitscore(b)
    time.sleep(0.1)

100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.41s/it]


In [39]:
# A function to remove BLAST entry results for same gene comparison by taking the highest bit score 
def remove_duplicates(temp):
    df=pd.DataFrame(temp,columns=["query_id","sub_id","bitscore"])
    d = df.sort_values("bitscore", ascending=False)
    d = d.drop_duplicates(['query_id'])
    d.reset_index(drop=True, inplace=True)
    
    return d

In [40]:
for a,b in l.items():
    print(b)


['CCPDPMNJ_01747', 'CCPDPMNJ_01748', 'CCPDPMNJ_01749', 'CCPDPMNJ_01750', 'CCPDPMNJ_01751', 'CCPDPMNJ_01752', 'CCPDPMNJ_01753', 'CCPDPMNJ_01754', 'CCPDPMNJ_01755', 'CCPDPMNJ_01756', 'ERR1007500(tet(45))S_75.88', 'CCPDPMNJ_01758', 'CCPDPMNJ_01759']
['MOOCJHBJ_01764', 'MOOCJHBJ_01765', 'MOOCJHBJ_01766', 'MOOCJHBJ_01767', 'MOOCJHBJ_01768', 'MOOCJHBJ_01769', 'MOOCJHBJ_01770', 'MOOCJHBJ_01771', 'MOOCJHBJ_01772', 'MOOCJHBJ_01773', 'ERR1007501(tet(45))S_75.88', 'MOOCJHBJ_01775', 'MOOCJHBJ_01776']
['CAEBHLLJ_02431', 'CAEBHLLJ_02432', 'CAEBHLLJ_02433', 'CAEBHLLJ_02434', 'ERR1036024(tet(45))S_76.35', 'CAEBHLLJ_02436', 'CAEBHLLJ_02437', 'CAEBHLLJ_02438', 'CAEBHLLJ_02439', 'CAEBHLLJ_02440', 'CAEBHLLJ_02441', 'CAEBHLLJ_02442', 'CAEBHLLJ_02443', 'CAEBHLLJ_02444', 'CAEBHLLJ_02445']
['HMEHPJPD_02238', 'ERR1036025(tet(45))S_76.22', 'HMEHPJPD_02240', 'HMEHPJPD_02241', 'HMEHPJPD_02242', 'HMEHPJPD_02243', 'HMEHPJPD_02244', 'HMEHPJPD_02245', 'HMEHPJPD_02246']
['LDJMFEKO_02617', 'ERR1036028(tet(45))S_76.0', 

In [41]:
for k, l in Drug_singleinstance_range_10_locus_dict10.items():
    print(l)
    #break
    for genome1 in Single_instance_drugclass_genomedict[k].keys():
        u =Single_instance_drugclass_genomedict[k][genome1]
        print(u)
        #break
     

{'ERR1007500(tet(45))S_75.88': ['CCPDPMNJ_01747', 'CCPDPMNJ_01748', 'CCPDPMNJ_01749', 'CCPDPMNJ_01750', 'CCPDPMNJ_01751', 'CCPDPMNJ_01752', 'CCPDPMNJ_01753', 'CCPDPMNJ_01754', 'CCPDPMNJ_01755', 'CCPDPMNJ_01756', 'ERR1007500(tet(45))S_75.88', 'CCPDPMNJ_01758', 'CCPDPMNJ_01759'], 'ERR1007501(tet(45))S_75.88': ['MOOCJHBJ_01764', 'MOOCJHBJ_01765', 'MOOCJHBJ_01766', 'MOOCJHBJ_01767', 'MOOCJHBJ_01768', 'MOOCJHBJ_01769', 'MOOCJHBJ_01770', 'MOOCJHBJ_01771', 'MOOCJHBJ_01772', 'MOOCJHBJ_01773', 'ERR1007501(tet(45))S_75.88', 'MOOCJHBJ_01775', 'MOOCJHBJ_01776'], 'ERR1036024(tet(45))S_76.35': ['CAEBHLLJ_02431', 'CAEBHLLJ_02432', 'CAEBHLLJ_02433', 'CAEBHLLJ_02434', 'ERR1036024(tet(45))S_76.35', 'CAEBHLLJ_02436', 'CAEBHLLJ_02437', 'CAEBHLLJ_02438', 'CAEBHLLJ_02439', 'CAEBHLLJ_02440', 'CAEBHLLJ_02441', 'CAEBHLLJ_02442', 'CAEBHLLJ_02443', 'CAEBHLLJ_02444', 'CAEBHLLJ_02445'], 'ERR1036025(tet(45))S_76.22': ['HMEHPJPD_02238', 'ERR1036025(tet(45))S_76.22', 'HMEHPJPD_02240', 'HMEHPJPD_02241', 'HMEHPJPD_0224

Index: []
Empty DataFrame
Columns: [query_id, sub_id, PI, bitscore, normalized_bitscore]
Index: []
Empty DataFrame
Columns: [query_id, sub_id, PI, bitscore, normalized_bitscore]
Index: []
Empty DataFrame
Columns: [query_id, sub_id, PI, bitscore, normalized_bitscore]
Index: []
Empty DataFrame
Columns: [query_id, sub_id, PI, bitscore, normalized_bitscore]
Index: []
Empty DataFrame
Columns: [query_id, sub_id, PI, bitscore, normalized_bitscore]
Index: []
Empty DataFrame
Columns: [query_id, sub_id, PI, bitscore, normalized_bitscore]
Index: []
Empty DataFrame
Columns: [query_id, sub_id, PI, bitscore, normalized_bitscore]
Index: []
Empty DataFrame
Columns: [query_id, sub_id, PI, bitscore, normalized_bitscore]
Index: []
Empty DataFrame
Columns: [query_id, sub_id, PI, bitscore, normalized_bitscore]
Index: []
Empty DataFrame
Columns: [query_id, sub_id, PI, bitscore, normalized_bitscore]
Index: []
Empty DataFrame
Columns: [query_id, sub_id, PI, bitscore, normalized_bitscore]
Index: []
Empty DataF

In [ ]:
single_similarity_array_dict={}
for k,l in tqdm(Drug_singleinstance_range_10_locus_dict10.items()):
    similarity_array=[]
    for genome1 in tqdm(Single_instance_drugclass_genomedict[k].keys()):  
        u=Single_instance_drugclass_genomedict[k][genome1]
        for genome2,b in l.items():
            temp=[]
            sum1=0
            sum2=0
            u.reset_index(drop=True, inplace=True)
            
            for index in range (len(u)):
                if u["sub_id"][index] in b:
                    temp.append((u["query_id"][index],u["sub_id"][index],u["normalized_bitscore"][index]))
                
            tempframe=remove_duplicates(temp)
            sum1=round(tempframe["bitscore"].sum(),3)
            
            if Single_contig_end_flag_dict[k][genome1] ==1:
                length_original_neighborhood=len(Drug_singleinstance_range_10_locus_dict10[k][genome1])
                difference=length_original_neighborhood-len(tempframe)
                sum2= sum1+(21-difference-len(tempframe))
                
            elif Single_contig_end_flag_dict[k][genome2]==1:
                length_original_neighborhood=len(Drug_singleinstance_range_10_locus_dict10[k][genome2])
                difference=length_original_neighborhood-len(tempframe)                                 
                
                sum2= sum1+(21-difference-len(tempframe))
            else:
                sum2=sum1
                #time.sleep(0.1)
           
            similarity_array.append((genome1,genome2,round(sum2,3)))
            time.sleep(0.1)
    
    single_similarity_array_dict[k]=similarity_array
    

 21%|███████▉                              | 95/455 [1:13:06<4:37:01, 46.17s/it]


 42%|███████████████▌                     | 192/455 [2:27:51<3:22:45, 46.26s/it]


 64%|███████████████████████▌             | 289/455 [3:42:36<2:07:57, 46.25s/it]


 81%|██████████████████████████████       | 369/455 [4:44:19<1:06:19, 46.27s/it]

In [ ]:
# A function to convert the similarity matrix to symmteric matric matrix by taking the average of scores
def converttosymmetrix(j):
    newj=[]
    
    for index1 in range(len(j)):
        
        score1=j[index1][2]
        for index2 in range(len(j)):
            if j[index2][1]==j[index1][0] and j[index2][0]==j[index1][1]:
                avg=round((j[index1][2]+j[index2][2])/2,3)
        newj.append((j[index1][0],j[index1][1],avg))
    return newj

In [ ]:
# Use the function to convert to symmetric matrix and store the final matrix in separate dictionaries
Single_instance_matrixframe={}
for i,j in tqdm(single_similarity_array_dict.items()):
    new=converttosymmetrix(j)
    finalresult=pd.DataFrame(new,columns=["Query_id","Sub_id","normalized_bitscore"])
    sim_mat=pd.crosstab(index=finalresult.iloc[:,0], columns=finalresult.iloc[:,1],values=finalresult.iloc[:,2], aggfunc=lambda x: x,colnames=None)
    Y=sim_mat.values
    Single_instance_matrixframe[i]=sim_mat
    time.sleep(0.1)


In [ ]:
# Generate a distance matrix and generate UPGMA clusters 
#os.mkdir("Outputs/UPGMA_single_clusters")
save_path="Outputs/UPGMA_single_clusters"
dataarray=[]

In [ ]:
hierarchy.set_link_color_palette(['r', 'g', 'y', 'm'])
matplotlib.rcParams['lines.linewidth'] = 1
for i,j in Single_instance_matrixframe.items(): 
    if len(j)>1:

        df = pd.DataFrame(j,columns=j.keys())
        r  = df.values
        df = df.transform(lambda x: 1 - x/r.max() )
        df = df.round(decimals=3)
        Y = df.values
        np.fill_diagonal(Y, 0)
        Y = distance.squareform(Y)
        fig = plt.figure(figsize=(30,5))
        Z = hierarchy.linkage(Y, 'average')
        R=dendrogram(Z, leaf_rotation=90, leaf_font_size=8, labels=j.keys())
        savename=os.path.join(save_path,i+"_.png")
        plt.ylabel("Distance between neighborhoods")
        plt.xlabel("Neighborhoods with their respective Genome IDs")
        plt.savefig(savename,bbox_inches='tight', dpi=100)
        plt.close()
        